## Find Majority Prediction

In [2]:
import psycopg2
import pandas as pd
from collections import Counter

In [ ]:
# Conect to the database
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="")
cur = conn.cursor()

In [ ]:
# Set the table names
parcels_table = "aoi2020"
bs_tensorflow = "bs_tensorflow"

# Set the folder to store the data
data_folder = ''

In [ ]:

# This script determines which is the majority class in the predictions from each run
# it updated the database table *_tf_results.

insertSql = f"SELECT pid::int, p0::int, p1::int, p2::int, p3::int FROM {bs_tensorflow}"

updateSql = "UPDATE {} set majclass = {}, majcount = {} WHERE pid = {}"

df = pd.read_sql(insertSql, conn)

r_index = df.columns[1:]

df['majclass'] = df.apply(lambda x: Counter(x[r_index]).most_common(1)[0][0], axis=1)
df['majcount'] = df.apply(lambda x: Counter(x[r_index]).most_common(1)[0][1], axis=1)

if len(df):
    for i in df.index:
        r = df.loc[i]
        cur.execute(updateSql.format(bs_tensorflow, r[-2], r[-1], r[0]))
    
    conn.commit()

    

In [ ]:
# Close database connection
database.close_connection()